# Keras-ImageDataGenerator

#This model contains a modified version of Keras ImageDataGenerator. It generate batches of tensor with real-time data augmentation. This generator is implemented for foreground segmentation or semantic segmentation.
# Please refer to https://keras.io/preprocessing/image/#image-preprocessing for more details

# Loading All library 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers import Activation, Dropout, Flatten, Dense
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
import cv2
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as plt
from keras.preprocessing import image


from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


# # Initialising the CNN


In [ ]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))


# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))




# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 5, activation = 'sigmoid'))
classifier.compile (optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


# Setting class_mode=None, it returns a tensor of (image, label).

# Initialize paths where images flow from.
#Create train_datagen and test set objects:


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/Users/abdulazizalreshidi/Desktop/sample_images',
                                                 target_size = (64, 64),classes=['anger_sad', 'casual', 'joy_happy', 'sorrow', 'surprise'],
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/Users/abdulazizalreshidi/Desktop/sample_images_vali',
                                            target_size = (64, 64),
                                            batch_size = 55,
                                            class_mode = 'categorical')

In [ ]:
num_of_train_samples=203
num_of_test_samples = 55
batch_size = 32

# Fit the generator:



In [ ]:
history=classifier.fit_generator(training_set,
                    steps_per_epoch=num_of_train_samples // batch_size,
                    epochs=70,
                    validation_data=test_set,
                    validation_steps=num_of_test_samples // batch_size)


# Confution Matrix and Classification Report


In [ ]:
Y_pred = classifier.predict_generator(test_set, num_of_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['Cats', 'Dogs', 'Horse']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

In [ ]:
Evaluate = classifier.evaluate_generator(test_set,steps=1)
print(Evaluate)

In [ ]:

import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# summarize history for loss


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# save the model



In [ ]:
classifier.save('CNN model 2.h5')


# Loding model


In [ ]:
from keras.models import load_model

new_model= load_model('/Users/abdulazizalreshidi/PycharmProjects/AI_Emotional Recognition/CNN model 2.h5')

new_model.compile (optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from sklearn.metrics import confusion_matrix
pred = classifier.predict_generator(training_set,steps=1,verbose=0)
pred = np.argmax(pred,axis = 1)
y_true = classifier.predict_generator(training_set,steps=1,verbose=0)

y_true = np.argmax(y_true,axis = 1)




In [ ]:

prediction=classifier.predict_generator(test_set,steps=1,verbose=0)
prediction = np.argmax(prediction,axis = 1)

nb_train_samples = 203
nb_validation_samples = 55
epochs = 70
batch_size = 32






Y_pred = classifier.predict_generator(test_set, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
a=test_set.classes
print('Confusion Matrix')
CM=confusion_matrix(a, prediction)
print(CM)
print('Classification Report')
target_names = ['anger_sad', 'casual', 'joy_happy', 'sorrow', 'surprise']
print(classification_report(y_true, pred))


In [ ]:
import pandas as pd
#ROC Curve
yp_dum=pd.get_dummies(a)
yt_dum=pd.get_dummies(prediction)
class_names=np.unique(np.array(CM)) #Used for Confusion Matrix visual.

# Plot all ROC curves
# Compute ROC curve and ROC area for each class

In [ ]:
import matplotlib.pyplot as plt

# Plot all ROC curves
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(yt_dum[i], yp_dum[i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
colors=["blue","darkviolet","magenta","orange","gold"]
emote_list=np.unique(np.array(train_datagen))
for i, color, emotion in zip(range(len(class_names)), colors, emote_list):
    plt.plot(fpr[i], tpr[i], color=color,
             label='ROC curve of class %s (area = %.2f)' %(emotion, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()



In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_true, pred, normalize=True)
plt.show ()